In [ ]:
# from xml.dom.minidom import parse, parseString

# import urllib
# from urllib.request import urlopen, Request, build_opener, HTTPCookieProcessor
# from urllib import urlencode

# import requests

# import mechanicalsoup
# from bs4 import BeautifulSoup
# from http.cookiejar import CookieJar

from selenium import webdriver

from itertools import product, islice

# from sys import getsizeof

from time import sleep

from collections import OrderedDict

import re

In [ ]:
url = 'https://compendium.ch/identa/searchPills.aspx?Platform=Desktop'

In [ ]:
driver = webdriver.Firefox()
driver.delete_all_cookies()
driver.get(url)

In [ ]:
# driver.find_elements_by_xpath('//*[@id]')
drop_down_menus_xpath = \
['//*[@id="ctl00_MainContent_ddlGalForm"]',
 '//*[@id="ctl00_MainContent_ddlGeoForm"]',
 '//*[@id="ctl00_MainContent_ddlColor"]',
 '//*[@id="ctl00_MainContent_ddlNotch"]', 
 '//*[@id="ctl00_MainContent_ddlLength"]']

click = lambda item_id, d=driver: d.find_element_by_id(item_id).click()
search_id = 'ctl00_MainContent_btnSearch'
reset_id = 'ctl00_MainContent_btnReset'
# res_table_id = 'ctl00_MainContent_gvwPills'
# res_table_id = 'ctl00_MainContent_gvwPills_ctl02_lblDescr'  # first row, text cell?
product_list_left_id = 'listProducts'
product_details_ids = ['ctl00_MainContent_ucProductDetail1_dvProduct_lblProductDescr', 
                       'ctl00_MainContent_ucProductDetail1_dvProduct_lblAtcDescr', 
                       'ctl00_MainContent_ucProductDetail1_dvProduct_lblAtcCode', 
                       'ctl00_MainContent_ucProductDetail1_dvProduct_lblKCH']



# search_button = lambda d=driver: d.find_element_by_id('ctl00_MainContent_btnSearch').click()
# reset_button = lambda d=driver: d.find_element_by_id('ctl00_MainContent_btnReset').click()
# //*[@id="ctl00_MainContent_ddlGalForm"]/option[1]

# check if no results returned
no_results = lambda: \
    len(driver.find_elements_by_id('ctl00_MainContent_lblNoDataFound')) != 0
# map from drop-down menu options to the correct range,
# to be used in the option's xpath
# special case for length menu:
# start range from 1, to check if there are no results (then skip)
drop_down_index = lambda x, num: list(range(1, len(x) + 2)) \
    if len(x) == num else \
    list(range(2, len(x) + 2))

find_digits = lambda s: [int(num) for num in re.findall('\d+', s)]

top_left_panel_digits = lambda: \
    find_digits(
        driver.find_element_by_id(
            'ctl00_MainTitle_lblTitle').text)

too_many_res = lambda: \
    len(top_left_panel_digits()) > 0 and \
    top_left_panel_digits()[0] >= 100
    
click_results_table = lambda d=driver: \
    d.find_element_by_xpath('//*[@id="ctl00_MainContent_gvwPills"]/tbody/tr[1]/td[2]')\
    .click()

In [ ]:
def log_results(d=driver, 
                product_list_left_id=product_list_left_id, 
                product_details_ids=product_details_ids):
    sleep(0.2)

    # click on results table
    click_results_table()
#         driver.find_element_by_class_name('gridview pills-grid').click()
    
    sleep(2)

    for num in range(top_left_panel_digits()[1]):
        # get product links (updates after each click / fetch)
        product_list_left = d.find_element_by_id(product_list_left_id)
        product_links = product_list_left.find_elements_by_tag_name('a')
        if 'num_of_products' not in globals():
            num_of_products =  len(product_links)
            assert num_of_products == top_left_panel_digits()[1]

#         sleep(3)

        product_links[num].click()

        sleep(3)

        print(
        OrderedDict(
            [(item_id, driver.find_element_by_id(item_id).text) 
             for item_id in product_details_ids])
        )

In [ ]:
# get the id's from the xpaths
menu_ids = [elem.partition('"')[-1].rpartition('"')[0] 
            for elem in drop_down_menus_xpath]
# make sure
assert set(menu_ids) == set(drop_down_menu_vals.keys())

In [ ]:
# populate all dropdown menu options
drop_down_menu_vals = {}
for xpath in drop_down_menus_xpath:
    elem = driver.find_elements_by_xpath(xpath)
    assert len(elem) == 1, 'Item shuold be unique.'
    elem = elem[0]
    values = [i.text # i.get_attribute('value') 
              for i in elem.find_elements_by_tag_name('option') 
              if i.text != 'Alle']
    drop_down_menu_vals[
        elem.get_attribute('id')] = values
    
# number of elements in the "length" menu
length_menu_num = \
    len(drop_down_menu_vals['ctl00_MainContent_ddlLength'])

In [ ]:
driver.get(url)
click(reset_id)

In [ ]:
menu_combinations = \
    [drop_down_index(drop_down_menu_vals[key], 
                     length_menu_num)
     for key in menu_ids]

# create all Cartesian products
menu_combinations = \
product(*menu_combinations)
# enumerate for logging purposes
menu_combinations = enumerate(menu_combinations)

In [ ]:
# an emaple of how all of the dropdown menus combinations might be clicked on
driver.implicitly_wait(2)  # seconds
# Set the amount of time to wait for a page load to complete before throwing an error
# driver.set_page_load_timeout(2.0)
# Set the amount of time that the script should wait during an
# execute_async_script call before throwing an error
# driver.set_script_timeout(2.0)

start_itr = 392
stop_itr = 500
iterable = islice(menu_combinations, 
                  start_itr, 
                  stop_itr)
for itr in iterable:
    click(reset_id)
#     reset_button()
    
    sleep(0.5)
    
#     for elem_id in menu_ids:
#         wait_for_elem(driver, elem_id)
    
    step, cart_prod = itr
    # set the drop-down menu state
    for j in range(len(cart_prod)):
        xpath = drop_down_menus_xpath[j]+'/option[{}]'.\
            format(cart_prod[j])
        driver.find_element_by_xpath(xpath).click()
            
    print('step number {}'.format(step))
#     print('product is {}'.format(list(zip(menu_ids, 
#                                      cart_prod))))

    # perform search
    click(search_id)
#     search_button()
    
    sleep(0.2)
#     wait_for_elem(driver, 'ctl00_MainContent_btnSearch')
    
    
    if cart_prod[j] == 1:  # if lengths = "Alle" 
        if no_results():  # returns no results
            # it is safe to skip all iterations on lengths menu
            try:
                [iterable.__next__() 
                 for _ in range(length_menu_num)]
            except:
                # if we've exceeded the iterator's length
                # it's still ok, since the next "islice batch"
                # would not lose any iteration with information
                pass
        elif too_many_res():  # returns too many results
            continue  # iterate over lengths in the length menu
        else:
            log_results()
            driver.get(url)
            sleep(0.2)
#     elif no_results():
#         pass
    else:
        if no_results():
            continue
        else:
            log_results()
            driver.get(url)
            sleep(0.2)

In [ ]:
print('done')

In [ ]:
clickable = driver.find_element_by_class_name('clickable')

In [ ]:
clickable.find_elements_by_tag_name('td')[1].click()

In [ ]:
click(res_table_id)

In [ ]:
product_list_left = driver.find_element_by_id(product_list_left_id)
product_links = product_list_left.find_elements_by_tag_name('a')
num_of_products = len(product_links)

In [ ]:
product_links[0].click()

In [ ]:
OrderedDict(
    [(item_id, driver.find_element_by_id(item_id).text) 
     for item_id in product_details_ids])

In [ ]:
product_list_left = driver.find_element_by_id(product_list_left_id)
product_links = product_list_left.find_elements_by_tag_name('a')
num_of_products = len(product_links)

In [ ]:
product_links[1].click()

In [ ]:
'num_of_products' not in globals()

In [ ]:
too_many_res()

In [ ]:
reset_button()

In [ ]:
search_button()

In [ ]:
driver.close()

In [ ]:
br = mechanicalsoup.StatefulBrowser()
br.set_verbose(2)

# page = br.open(url)
page = br.open(url)
form = page.soup.form

# br.get(url)

# br.  # ('select#ctl00_MainContent_ddlGalForm') #, {“class”:”signin”})

drop_down_menu = dict(
    (menu['name'], 
     [opt['value'] for opt in menu.find_all('option') 
      if opt.text != 'Alle']) 
 for menu in form.find_all('select'))

# br.select_form('select', {'name': 'ctl00$MainContent$ddlGalForm'})
# br.select_form('table#ctl00_MainContent_tblCriterias')
# br.select_form('select#ctl00$MainContent$ddlGalForm')

# form.find('select#ctl00$MainContent$ddlGalForm', {"name": "size", "value": "medium"})["checked"] = ""
# form.find("select", {"name": "size", "value": "medium"})["checked"] = ""

# form.find('select', {'id': 'ctl00_MainContent_ddlGalForm'}).get('value', '')
# form.find('select', {'id': 'ctl00_MainContent_ddlGalForm'})#['option selected']
# form.find('select', {'id': 'ctl00_MainContent_ddlGalForm'}).find_control

# br.new_control(type='identaDropdown', name='ctl00_MainContent_ddlGalForm', value='')

# sf = util.system_file()
# br.select_form("#biqmac")
# br["mail"] = sf.mail
# br["name"] = sf.cluster_user
# br["graphname"] = core.filename(biqmac_file)
# br["graphfile"] = biqmac_file
# br["source"] = "Custom Generator"

# br.get_current_form().set("submit", "mac", force=True)
# res = br.submit_selected()


# br.launch_browser()

# br.new_control(type='select', 
#                name='ctl00$MainContent$ddlGalForm', 
#                value='Alle')

# br.get_current_form().set()

# br._build_request()

# br.select_form('select#ctl00_MainContent_ddlGalForm.identaDropdown')

In [ ]:
key = list(drop_down_menu.keys())[1]
print(key)
# br._build_request(form).data[key] = drop_down_menu[key][1]

In [ ]:
drop_down_menu

In [ ]:
br._build_request(form).data[key] = drop_down_menu[key][1]

In [ ]:
for k,v in br._build_request(form).data.items():
    print(k, v)
# br._build_request(form).data['']

In [ ]:
cj = CookieJar()
opener = build_opener(HTTPCookieProcessor(cj))

# input-type values from the html form
# formdata = { "username" : username, "password": password, "form-id" : "1234" }
# data_encoded = urllib.urlencode(formdata)
# response = opener.open("https://page.com/login.php", data_encoded)

response = opener.open(url)
html = response.read()

# html

In [ ]:
soup = BeautifulSoup(html, 'html5lib')
soup

In [ ]:
# help(soup.find_all('select')[0])

# soup.find_all('select')[0].find_all('option')[0].text

# soup.select('select')

# soup.find_all('select')[0].get('tabindex')

In [ ]:
# drop_down_menu = dict(
#     (menu.get('tabindex'), 
#      [opt.text for opt in menu.find_all('option') 
#       if opt.text != 'Alle']) 
#  for menu in soup.find_all('select'))

drop_down_menu = dict(
    (menu['name'], 
     [opt.text for opt in menu.find_all('option') 
      if opt.text != 'Alle']) 
 for menu in soup.find_all('select'))

In [ ]:
drop_down_menu.keys()

In [ ]:
data = {
    'ctl00$MainContent$ddlLength': drop_down_menu['ctl00$MainContent$ddlLength'],
    'ctl00$ctl00$txtSearchWord': '',
    '__EVENTTARGET': soup.find('input', {'name': '__EVENTTARGET'}).get('value', ''),
    '__EVENTARGUMENT': soup.find('input', {'name': '__EVENTARGUMENT'}).get('value', ''),
    '__LASTFOCUS': soup.find('input', {'name': '__LASTFOCUS'}).get('value', ''),
    '__VIEWSTATE': soup.find('input', {'name': '__VIEWSTATE'}).get('value', ''),
    '__VIEWSTATEGENERATOR': soup.find('input', {'name': '__VIEWSTATEGENERATOR'}).get('value', ''),
    '__EVENTVALIDATION': soup.find('input', {'name': '__EVENTVALIDATION'}).get('value', ''),
}

In [ ]:
# from bs4 import BeautifulSoup
import requests

url = 'https://compendium.ch/identa/searchPills.aspx'
cj = CookieJar()

with requests.Session() as session:
    session.headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/40.0.2214.115 Safari/537.36'}

    # parsing parameters
    response = session.get(url, cookies=cj)
    soup = BeautifulSoup(response.content, 'html5lib')
    
#     print(soup)
    
    drop_down_menu = dict(
        (menu['name'], 
         [opt.text for opt in menu.find_all('option') 
          if opt.text != 'Alle']) 
        for menu in soup.find_all('select'))
    
    hidden_inputs = soup.find_all('input', {"type" : "hidden"})
        
    data = {
        'ctl00$MainContent$ddlLength': drop_down_menu['ctl00$MainContent$ddlLength'][0],
        'ctl00$ctl00$txtSearchWord': '',
        '__EVENTTARGET': soup.find('input', {'name': '__EVENTTARGET'}).get('value', ''),
        '__EVENTARGUMENT': soup.find('input', {'name': '__EVENTARGUMENT'}).get('value', ''),
#         'ctl00_RadScriptManager1_TSM': soup.find('input', {'name': 'ctl00_RadScriptManager1_TSM'}).get('value', ''),
#         'ctl00_RadStyleSheetManager1_TSSM': soup.find('input', {'name': 'ctl00_RadStyleSheetManager1_TSSM'}).get('value', ''),
#         '__LASTFOCUS': soup.find('input', {'name': '__LASTFOCUS'}).get('value', ''),
#         '__LASTFOCUS': soup.find('input', {'name': '__LASTFOCUS'}).get('value', ''),
#         '__LASTFOCUS': soup.find('input', {'name': '__LASTFOCUS'}).get('value', ''),
#         '__LASTFOCUS': soup.find('input', {'name': '__LASTFOCUS'}).get('value', ''),
#         '__LASTFOCUS': soup.find('input', {'name': '__LASTFOCUS'}).get('value', ''),
#         '__LASTFOCUS': soup.find('input', {'name': '__LASTFOCUS'}).get('value', ''),
#         '__LASTFOCUS': soup.find('input', {'name': '__LASTFOCUS'}).get('value', ''),
        '__VIEWSTATE': soup.find('input', {'name': '__VIEWSTATE'}).get('value', ''),
        '__VIEWSTATEGENERATOR': soup.find('input', {'name': '__VIEWSTATEGENERATOR'}).get('value', ''),
        '__EVENTVALIDATION': soup.find('input', {'name': '__EVENTVALIDATION'}).get('value', ''),
    }
    
    for hidden in hidden_inputs:
        try:
            data[hidden] = soup.find('input', {'name': hidden}).get('value', '')
        except:
            pass

    # parsing data
    response = session.post(url, data=data, cookies=cj)

    soup = BeautifulSoup(response.content, 'html5lib')
    
#     print(soup)
#     print(data)
    
# #     for row in soup.select('table.tData01 tr'):
#     for row in soup.select('table.tData01 tr'):
#         print([td.text for td in row.find_all('td')])
#         print(soup.select('table.tData01 tr'))


In [ ]:
soup

In [ ]:
from bs4 import BeautifulSoup
import requests

url = 'http://www.koreabaseball.com/Record/Player/HitterDetail/Game.aspx?playerId=76325'

# with requests.Session() as session:
#     session.headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/40.0.2214.115 Safari/537.36'}

    # parsing parameters
    response = session.get(url)
    soup = BeautifulSoup(response.content, 'html5lib')
    
#     print(soup)
    
    data = {
        'ctl00$ctl00$cphContainer$cphContents$ddlYear': '2013',
        'ctl00$ctl00$txtSearchWord': '',
        '__EVENTTARGET': soup.find('input', {'name': '__EVENTTARGET'}).get('value', ''),
        '__EVENTARGUMENT': soup.find('input', {'name': '__EVENTARGUMENT'}).get('value', ''),
        '__LASTFOCUS': soup.find('input', {'name': '__LASTFOCUS'}).get('value', ''),
        '__VIEWSTATE': soup.find('input', {'name': '__VIEWSTATE'}).get('value', ''),
        '__VIEWSTATEGENERATOR': soup.find('input', {'name': '__VIEWSTATEGENERATOR'}).get('value', ''),
        '__EVENTVALIDATION': soup.find('input', {'name': '__EVENTVALIDATION'}).get('value', ''),
    }

    # parsing data
    response = session.post(url, data=data)

    soup = BeautifulSoup(response.content, 'html5lib')
    
#     print(soup)
    
#     for row in soup.select('table.tData01 tr'):
    for row in soup.select('table.tbl tr'):
        print([td.text for td in row.find_all('td')])
        print(soup.select('table.tData01 tr'))


In [ ]:
from bs4 import BeautifulSoup
import requests

url = 'http://www.koreabaseball.com/Record/Player/HitterDetail/Game.aspx?playerId=76325'

with requests.Session() as session:
    session.headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/40.0.2214.115 Safari/537.36'}

    # parsing parameters
    response = session.get(url)
    soup = BeautifulSoup(response.content, 'html5lib')
    
#     print(soup)
    
    data = {
        'ctl00$ctl00$cphContainer$cphContents$ddlYear': '2013',
        'ctl00$ctl00$txtSearchWord': '',
        '__EVENTTARGET': soup.find('input', {'name': '__EVENTTARGET'}).get('value', ''),
        '__EVENTARGUMENT': soup.find('input', {'name': '__EVENTARGUMENT'}).get('value', ''),
        '__LASTFOCUS': soup.find('input', {'name': '__LASTFOCUS'}).get('value', ''),
        '__VIEWSTATE': soup.find('input', {'name': '__VIEWSTATE'}).get('value', ''),
        '__VIEWSTATEGENERATOR': soup.find('input', {'name': '__VIEWSTATEGENERATOR'}).get('value', ''),
        '__EVENTVALIDATION': soup.find('input', {'name': '__EVENTVALIDATION'}).get('value', ''),
    }

    # parsing data
    response = session.post(url, data=data)

    soup = BeautifulSoup(response.content, 'html5lib')
    
#     print(soup)
    
#     for row in soup.select('table.tData01 tr'):
    for row in soup.select('table.tbl tr'):
        print([td.text for td in row.find_all('td')])
        print(soup.select('table.tData01 tr'))


In [ ]:
soup.select('table.tData01 tr')

In [ ]:
soup = BeautifulSoup(data).read()
# BeautifulSoup(YOUR_MARKUP, "html5lib")

In [ ]:
import urllib.request

request = urllib.request.Request(url)
response = urllib.request.urlopen(request)
the_page = response.read()
theText = the_page.decode()
soup = BeautifulSoup(theText, 'html5lib')

In [ ]:
print(soup.prettify())

In [ ]:
soup.findAll(name='select')
# ctl00$MainContent$ddlGalForm
# 'ctl00_MainContent_ddlGalForm'
# 'identaDropdown'
# <option value="">Alle</option>
# <option selected="selected" value="SF">Andere</option>
# select


In [ ]:
html = response.read()

soup = BeautifulSoup(html, 'html5lib')

# numbers = soup.select("option")
numbers = soup.select('ctl00_MainContent_ddlGalForm')
print(numbers)

for number in numbers:
    number = number["value"]
    response = self.opener.open(page+"plan?utf8=&current_subscriber="+number, None, 10)
    print(response)
    
# response

In [ ]:
# import string, sys
from bs4 import BeautifulSoup

changable_url = 'http://www.asusparts.eu/partfinder/Asus/All%20In%20One/E%20Series'
page = urlopen(changable_url)
base_url = 'http://www.asusparts.eu'
soup = BeautifulSoup(page)

redirects = []
model_info = []

select = soup.find(id='myselectListModel')
print(select.get_text())


options = select.findAll('option')

for option in options:
    if(option.has_attr('redirectvalue')):
       redirects.append(option['redirectvalue'])

for r in redirects:
    rpage = urlopen(base_url + r.replace(' ', '%20'))
    s = BeautifulSoup(rpage)
    print(s)


# sys.exit()

In [ ]:
import urllib.request
import urllib.parse
url="http://www.nasdaq.com/symbol/dal/historical"
pageInputs={"ddlTimeFrame":"2y"}
pageGets = urllib.parse.urlencode(pageInputs)
pageGets=pageGets.encode("ascii")
serReq=urllib.request.Request(url,pageGets)
opReq=urllib.request.urlopen(serReq)
rdReq=opReq.read()
dcReq=rdReq.decode()

In [ ]:
dcReq

In [ ]:
# data.read()


In [ ]:
from urllib.request import quote

# for r in redirects:
#     rpage = urlopen(urljoin(base_url, quote(r)))
#     rpage = urlopen(url)
#     s = BeautifulSoup(rpage)
#     print(s)

rpage = urlopen(url)

In [ ]:
dropdown_id = 'DropDownList1'
button_id = 'Button1'
table_id = 'table1'
# browser = webdriver.Chrome()
browser = webbrowser.Chrome()
url = 'https://compendium.ch/identa/searchPills.aspx'
# browser.get(url)
browser.open_new(url)

In [ ]:
# Data getting
dropdown = _browser.find_element_by_id(dropdown_id)
for option in dropdown.dropdown.find_elements_by_tag_name('option'):
    select = Select(browser.find_element_by_id(dropdown_id))
    select.select_by_value(option.text)
    button = browser.find_element_by_id(button_id)
    button.click()
    table = browser.find_element_by_id(table_id)
    html = table.get_attribute('outerHTML')
browser.close()